In [8]:
# import cv2
# import numpy 
# import numpy as np
# import time
# from skimage.feature import peak_local_max
# from skimage.morphology import watershed
# from scipy import ndimage
# from scipy import ndimage as ndi

# cap = cv2.VideoCapture("images/DIC-C2DH-HeLa/Sequence 1/t%03d.tif")
# #cap = cv2.VideoCapture("images/Fluo-N2DL-HeLa/Sequence 1/t%03d.tif")
# #cap = cv2.VideoCapture("images/PhC-C2DL-PSC/Sequence 1/t%03d.tif")

# while(1):
#     ret, frame = cap.read()
    
#     # some processing here?
#     if (ret) :
# #         shifted = cv2.pyrMeanShiftFiltering(frame, 41, 51)
# #         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# #         thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
# #         im = gray.copy()    
# #         D = ndimage.distance_transform_edt(thresh)
# #         localMax = peak_local_max(D, indices=False, min_distance=3,  labels=thresh)
# #         markers = ndimage.label(localMax, structure=numpy.ones((3, 3)))[0]
# #         labels = watershed(-D, markers, mask=thresh)
# #         print("[INFO] {} unique segments found".format(len(numpy.unique(labels)) - 1))

# #         conts=[]    
# #         for label in numpy.unique(labels):
# #     # if the label is zero, we are examining the 'background'
# #     # so simply ignore it
# #             if label == 0:
# #                 continue
# # #     otherwise, allocate memory for the label region and draw
# #     # it on the mask
# #             mask = numpy.zeros(gray.shape, dtype="uint8")
# #             mask[labels == label] = 255
# #     # detect contours in the mask and grab the largest one
# #             cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
# #             c = max(cnts, key=cv2.contourArea)
# #             rect = cv2.minAreaRect(c)
# #             box = cv2.boxPoints(rect)
# #             box = numpy.int0(box)  
# #             if cv2.contourArea(c) > 150: 
# # #         cv2.drawContours(image,c,-1,(0,255,0))
# #                 cv2.drawContours(frame,[box],-1,(0,255,0))
    
    
    
#         cv2.imshow('image',frame)
#         time.sleep(0.5)
#     else:
#         cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
#     # 'q' to quit playback
#     if (cv2.waitKey(1) & 0xFF == ord('q')):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [17]:
# cap = cv2.VideoCapture("images/DIC-C2DH-HeLa/Sequence 1/t%03d.tif")
# ret,frame = cap.read()
# print(ret)
# cv2.imshow('image',frame)

True


In [9]:
import cv2
import numpy 
import numpy as np
import time
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
from scipy import ndimage as ndi

In [10]:
def contrast_strech(image):
    a = 0
    b = 255
    c = image.min()
    d_max = image.max()
    image_new = image
    image_new[:,:,:] = (image[:,:,:] - c)* ((b-a)/(d_max - c)) + a #(I-c)(b-a/max - min) + a
    return image_new

def convolve_max_filtered(image,N):
    h,w = image.shape
    output = np.zeros((h,w),dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 0)
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                max_pixel = np.amax((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= max_pixel
    output = (output*255).astype("uint8")
    return output


def convolve_min_filtered(image,N):
    h,w = image.shape
    output = np.full((h,w),255,dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 255)
    
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                min_pixel = np.amin((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= min_pixel
           
    output = (output*255).astype("uint8")
    return output


def remove_background(image,back_img):
    h,w = image.shape
    output = np.zeros((h,w),dtype = "uint8")
    output = np.subtract(image,back_img)
    return output

def run_algo(image,filter_size, M):
    if M == 0:
        max_filter =  convolve_max_filtered(image,filter_size)
        #min_filter =  convolve_min_filtered(max_filter,filter_size)
        final_image = remove_background(image, max_filter)
    else:
        min_filter = convolve_min_filtered(image,filter_size)
        #max_filter = convolve_max_filtered(min_filter,filter_size)
        final_image = remove_background(image, min_filter)
    return final_image

def image_sharpening(image):
    sigma = 10
    a = 10
    blur_image = cv2.GaussianBlur(image,(0,0),sigma)  #L = I*G
    #H = image - blur_image #H = I - L
    H = cv2.subtract(image,blur_image)
    final_image = H*a + image #o = I+H*a
    return final_image


def apply_watershed(img):
    img_array = img.copy()
    distance = ndi.distance_transform_edt(img_array)
    markers = ndi.label(peak_local_max(distance, indices = False, labels = img_array))[0]
    ws_labels = watershed(-distance, markers, mask = img_array)
    return ws_labels



In [11]:
#cap = cv2.VideoCapture("images/Fluo-N2DL-HeLa/Sequence 1/t%03d.tif")
cap = cv2.VideoCapture("images/PhC-C2DL-PSC/Sequence 1/t%03d.tif")
#cap = cv2.VideoCapture("images/Fluo-N2DL-HeLa/Sequence 1/t017.tif")
filter_size = 3
M = 1
while(1):
    ret, frame = cap.read()
    if (ret) :  
        cv2.imshow('image1',frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('image2',gray)
        #applying min_max filtering
        #frame = run_algo(gray, filter_size,M)
        #inverting the image
        frame = cv2.bitwise_not(255-gray)
        cv2.imshow('image3',frame)
        kernel_erosion = np.ones((3,3),np.uint8)
        
        erosion = cv2.erode(frame,kernel_erosion,iterations = 1)
        cv2.imshow('image4',erosion)
        #filterSize = (60,60)
        kernel_dilation = np.ones((5,5), np.uint8)
        img_dilation = cv2.dilate(erosion, kernel_dilation, iterations=1)
        
        cv2.imshow('image5',img_dilation)
       # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize)
       # opening = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
        #a = contrast_strech
        #frame = image_sharpening(frame)
        #
        #blur = cv2.GaussianBlur(frame,(5,5),0)
        th = cv2.adaptiveThreshold(img_dilation,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        cv2.imshow('image6',th)
        frame = remove_background(th,img_dilation)
        cv2.imshow('image7',frame)
        
        blur = cv2.GaussianBlur(frame,(5,5),0)
        ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        

        
        
        
        
        #frame = image_sharpening(frame)
        #frame = apply_watershed(frame)
        cv2.imshow('image',th)
        #cv2.imshow('image2',img_dilation)
        time.sleep(0.5)
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # 'q' to quit playback
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break
        
cap.release()
cv2.destroyAllWindows()